# pyBela Tutorial 3: Logger
This notebook is a tutorial for the Logger class in the pyBela python library. As opposed to the Streamer, the Logger stores variable values directly in binary files in the Bela board. This is more reliable than streaming data with the Streamer with the saving mode enabled, which depends on the websocket connection. The Logger will store the data in Bela even if the websocket connection is lost, and you can retrieve the data later. 

As with the previous tutorials, you will need to run the `potentiometers` project in Bela. If you haven't done it yet, copy the project onto Bela:

In [ ]:
!rsync -rv ../bela-code/potentiometers root@bela.local:Bela/projects

And compile and run the project using either the IDE or by running the following command in the Terminal:
```bash
ssh root@bela.local "make -C Bela stop Bela PROJECT=potentiometers run" 
```
(Running this on a jupyter notebook will block the cell until the program is stopped on Bela.)

You will need to connect the two potentiometers to the Bela analog inputs 0 and 1 (a schematic can be found in the `1_Streamer.ipynb` tutorial). 

Run the cells below to connect to the Logger to Bela and start logging data:

In [ ]:
import asyncio
import os
import pandas as pd
from pyBela import Logger

In [ ]:
logger = Logger()
logger.connect()

### Logging files with automatic transfer

Similarly to the Streamer and the Monitor, we can start and stop a logging session with `start_logging()` and `stop_logging`, and use asyncio to time the logging session:

In [ ]:
file_paths = logger.start_logging(
    variables=["pot1", "pot2"])
await asyncio.sleep(0.5)
logger.stop_logging()

### Loading the data from the binary file
The Logger automatically transfers the files from Bela to the computer whilst the logging session is happening. This avoids long waiting times at the end of the session. 

`start_logging()` returns `file_paths`, a dictionary containing the paths in Bela of the files generated and the local paths (in this computer) to which the files are copied. You can use these paths to automate the processing of the data:

In [ ]:
data = {}
for var in ["pot1", "pot2"]:
    data[var] = logger.read_binary_file(
        file_path=file_paths["local_paths"][var], timestamp_mode=logger.get_prop_of_var(var, "timestamp_mode"))

You might notice that the last buffer of each variable have 0.0 values at the end. This is because the buffers send by Bela to the Logger have a fixed size, and the last buffer might not be completely filled, so the remaining values are filled with 0.0. 

In [ ]:
data["pot1"]["buffers"][-1]

You can also flatten the buffers using:

In [ ]:
flatten_data = {}
for var in ["pot1", "pot2"]:
    flatten_data[var] = {"timestamps": [], "data": []}
    for _buffer in data[var]["buffers"]:
        flatten_data[var]["timestamps"].extend([_buffer["ref_timestamp"] + i for i in range(len(_buffer["data"]))])
        flatten_data[var]["data"].extend(_buffer["data"])
    
df = pd.DataFrame(flatten_data["pot1"])
df.head()

Whilst the Logger by default transfers the binary files to Bela, the files are not removed from Bela. You can remove the files from Bela with `delete_file_from_bela()`:

In [ ]:
for file in file_paths["remote_paths"].values():
    logger.delete_file_from_bela(file, verbose=True)

### Scheduling logging sessions
You can schedule a logging session to start and stop at a specific time using the `schedule_logging()` method. This method takes the same arguments as `start_logging()`, but it also takes a `timestamps` and `durations` argument.

In [ ]:
latest_timestamp = logger.get_latest_timestamp() # get the latest timestamp
sample_rate = logger.sample_rate # get the sample rate
start_timestamp = latest_timestamp + sample_rate # start logging 1 second after the latest timestamp
duration = sample_rate * 2 # log for 2 seconds

file_paths = logger.schedule_logging(
    variables=["pot1", "pot2"],
    timestamps=[start_timestamp, start_timestamp],
    durations=[duration, duration], 
    transfer=True, 
    logging_dir="./")


### Logging files without automatic transfer

Alternatively, you can set `transfer=False` in `start_logging()` and transfer the files manually with `copy_file_from_bela()`:

In [ ]:
file_paths =  logger.start_logging(
    variables= ["pot1", "pot2"], transfer=False)
await asyncio.sleep(0.5)
logger.stop_logging()

file_paths

Note that the dict in `local_paths` is empty: since we disabled the automatic transfer, no local paths have been assigned. We can now copy the files using `copy_file_from_bela()`:

In [ ]:
for var in ["pot1", "pot2"]:
    logger.copy_file_from_bela(remote_path=file_paths["remote_paths"][var], local_path=os.path.basename(file_paths["remote_paths"][var]))

You might end up with a few `.bin` files in Bela. You can either remove them one by one with `delete_file_from_bela()` as explained above, or remove all `.bin` files in the Bela project with `delete_all_bin_files_in_project()``

In [ ]:
logger.delete_all_bin_files_in_project()